In [4]:
# -*- coding: utf-8 -*-
"""
Generate Pseudo dataset of triangle types
Revised for better validity and diversity.
"""
import random
import time
import pandas as pd # Tidak digunakan di sini, tapi mungkin berguna untuk analisis nanti
import math

# --- Konfigurasi ---
max_side = 20       # Maksimum panjang sisi individual
max_data = 5000     # Jumlah data yang akan digenerate
file_name = "triangle_revised.csv" # Nama file output baru

# --- Definisi Tipe (Tidak berubah) ---
sides_cols = ["side1", "side2", "side3"]
side_types_list = ["equilateral", "isosceles", "scalene"]
angle_types_list = ["equiangular", "acute", "right", "obtuse"]


# --- Fungsi Helper Validitas ---
def is_triangle_valid(s1, s2, s3):
    """Mengecek apakah tiga sisi dapat membentuk segitiga yang valid."""
    # Pastikan semua sisi positif
    if not (s1 > 0 and s2 > 0 and s3 > 0):
        return False
    sides = sorted([s1, s2, s3])
    return sides[0] + sides[1] > sides[2]

# --- Fungsi Klasifikasi (Memastikan input terurut) ---
def type_by_side(t3_sorted): # Input HARUS sudah terurut
    if t3_sorted[0] == t3_sorted[1] == t3_sorted[2]:
        return 0 # equilateral
    elif (t3_sorted[0] == t3_sorted[1]) or (t3_sorted[1] == t3_sorted[2]):
        return 1 # isosceles
    else:
        return 2 # scalene

def type_by_angle(t3_sorted): # Input HARUS sudah terurut
    if t3_sorted[0] == t3_sorted[1] == t3_sorted[2]: # Equilateral juga Equiangular
         return 0 # equiangular
    
    # Gunakan hukum Pythagoras pada sisi terurut
    # c^2 = a^2 + b^2 (siku-siku)
    # c^2 < a^2 + b^2 (lancip)
    # c^2 > a^2 + b^2 (tumpul)
    # t3_sorted[2] adalah sisi terpanjang (hipotenusa potensial)
    s1_sq_plus_s2_sq = t3_sorted[0]**2 + t3_sorted[1]**2
    s3_sq = t3_sorted[2]**2
    
    # Gunakan toleransi kecil untuk perbandingan float jika sisi float
    # tapi karena kita pakai int, perbandingan langsung oke
    if s1_sq_plus_s2_sq > s3_sq:
        return 1 # acute
    elif s1_sq_plus_s2_sq < s3_sq:
        return 3 # obtuse
    else: # s1_sq_plus_s2_sq == s3_sq
        return 2 # right

# --- Generator Segitiga (Revisi) ---
def equilateral_triangle():
    side = random.randint(1, max_side)
    return [side, side, side]

def isosceles_triangle():
    while True:
        # Sisi yang sama (kaki)
        side_a = random.randint(1, max_side)
        # Sisi ketiga (alas), harus < 2 * side_a agar valid
        side_b = random.randint(1, max_side) 
        if side_b < 2 * side_a and side_b != side_a: # Pastikan bukan equilateral
             # Cek juga apakah side_a + side_b > side_a (selalu benar jika side_b > 0)
            tr = sorted([side_a, side_a, side_b])
            if is_triangle_valid(tr[0], tr[1], tr[2]):
                 return [side_a, side_a, side_b] # Kembalikan urutan asli sebelum sort untuk shuffle

def scalene_triangle():
    while True:
        side1 = random.randint(1, max_side)
        side2 = random.randint(1, max_side)
        side3 = random.randint(1, max_side)
        
        # Pastikan tidak sama sisi atau sama kaki
        if side1 == side2 or side1 == side3 or side2 == side3:
            continue # Coba lagi

        if is_triangle_valid(side1, side2, side3):
            return [side1, side2, side3]

def right_triangle():
    """Menghasilkan segitiga siku-siku dengan sisi integer hingga max_side."""
    attempts = 0
    while attempts < 100: # Batasi percobaan untuk menghindari infinite loop
        # Pilih dua sisi yang lebih pendek (kaki siku-siku)
        a = random.randint(1, max_side -1) # Kurangi 1 agar ada ruang untuk b
        b = random.randint(1, max_side -1) # Kurangi 1 agar ada ruang untuk a

        c_squared = a**2 + b**2
        c = math.isqrt(c_squared) # Akar kuadrat integer (Python 3.8+)
                                   # Jika Python < 3.8: c = int(math.sqrt(c_squared))

        # Cek apakah merupakan triplet Pythagoras dan c tidak melebihi max_side
        if c**2 == c_squared and c <= max_side:
            if is_triangle_valid(a,b,c) and a !=b and a!=c and b!=c: # Pastikan bisa jadi scalene right
                 return [a,b,c]
            # Bisa juga isosceles right, misal a=b
            elif is_triangle_valid(a,b,c) and a==b and a!=c:
                 return [a,b,c]

        attempts +=1
    # Jika gagal setelah banyak percobaan, fallback ke triplet terkenal (atau scalene)
    # Ini untuk memastikan fungsi selalu mengembalikan sesuatu
    # print("Fallback right_triangle")
    rt3_fallback = [[3,4,5],[5,12,13],[8,15,17]] # Pastikan sisi <= max_side
    valid_fallback = [t for t in rt3_fallback if t[2] <= max_side]
    if valid_fallback:
        return list(random.choice(valid_fallback))
    else: # Jika max_side terlalu kecil, buat scalene saja
        return scalene_triangle()


def generate_one_triangle_balanced_sides(): # Ubah nama fungsi atau buat baru
    """
    Mencoba menghasilkan jenis segitiga dasar (untuk sisi) dengan proporsi
    yang lebih seimbang. Tipe sudut tetap akan dihitung.
    """
    # Tentukan jenis segitiga dasar yang ingin digenerate secara bergantian atau acak merata
    # Kita akan pakai random.choice untuk memilih salah satu generator dasar
    # Ini akan memanggil setiap generator dasar dengan probabilitas yg sama
    
    # Kita akan targetkan lebih banyak right triangle juga untuk memastikan jumlahnya
    # Pilihan generator: 0=equilateral, 1=isosceles, 2=scalene, 3=right
    # Kita bisa atur proporsinya, misal:
    # Equilateral: ~25%
    # Isosceles:  ~25%
    # Scalene:    ~25% (bisa jadi acute/obtuse/right)
    # Right:      ~25% (khusus untuk memastikan ada yg right)

    generator_choice = random.randint(0, 3) # 0, 1, 2, 3

    if generator_choice == 0:
        tr = equilateral_triangle()
    elif generator_choice == 1:
        tr = isosceles_triangle()
    elif generator_choice == 2: # Scalene, bisa berbagai macam sudut
        tr = scalene_triangle()
    else: # generator_choice == 3, fokus generate right
        tr = right_triangle()

    if tr is None: # Fallback jika salah satu generator gagal
        # print("Fallback ke scalene karena generator utama gagal.")
        tr = scalene_triangle()

    tsorted = sorted(tr)
    
    tside_idx = type_by_side(tsorted)
    tangle_idx = type_by_angle(tsorted)
    
    random.shuffle(tr)
    return tr, tside_idx, tangle_idx

# --- Main Program (MODIFIKASI DI SINI) ---
current_time = time.time()
random.seed(current_time)

cangle_counts = [0] * len(angle_types_list)
cside_counts = [0] * len(side_types_list)

print(f"Memulai generasi data (target lebih seimbang untuk sisi) ke file: {file_name}")
with open(file_name, 'w') as file:
    header = "side1,side2,side3,side_type,angle_type"
    file.write(header + "\n")

    # Kita buat loop untuk setiap jenis dasar sisi agar jumlahnya mirip
    # Ini pendekatan kasar, karena hasil akhir side_type masih bisa bervariasi
    # tergantung validitas dan apakah right_triangle jadi scalene/isosceles
    
    # Target per jenis dasar (misal, jika max_data=5000, target ~1250 per jenis dasar)
    # Ini lebih ke "mencoba" memanggil generator dasar dengan frekuensi sama
    for i in range(max_data):
        if (i + 1) % 500 == 0:
            print(f"Memproses data ke-{i+1} dari {max_data}...")
            
        # Panggil fungsi generator yang sudah dimodifikasi
        generated_triangle, tside_idx, tangle_idx = generate_one_triangle_balanced_sides()
        
        side_label = side_types_list[tside_idx]
        angle_label = angle_types_list[tangle_idx]
    
        cside_counts[tside_idx] += 1
        cangle_counts[tangle_idx] += 1

        line = f"{generated_triangle[0]},{generated_triangle[1]},{generated_triangle[2]},{side_label},{angle_label}"
        file.write(line + "\n")

print("\n--- Generasi Data Selesai ---")
# ... (Bagian print distribusi sama seperti sebelumnya) ...
print("Distribusi Tipe Sisi (Counts):")
for i, stype in enumerate(side_types_list):
    print(f"- {stype}: {cside_counts[i]}")

print("\nDistribusi Tipe Sudut (Counts):")
for i, atype in enumerate(angle_types_list):
    print(f"- {atype}: {cangle_counts[i]}")

print(f"\nData telah disimpan di {file_name}")

Memulai generasi data (target sisi lebih merata) ke file: triangle_revised_5k.csv
Memproses data ke-500 dari 5000 (Total attempts: 500)...
Memproses data ke-1000 dari 5000 (Total attempts: 1000)...
Memproses data ke-1500 dari 5000 (Total attempts: 1500)...
Memproses data ke-2000 dari 5000 (Total attempts: 2000)...
Memproses data ke-2500 dari 5000 (Total attempts: 2500)...
Memproses data ke-3000 dari 5000 (Total attempts: 3000)...
Memproses data ke-3500 dari 5000 (Total attempts: 3500)...
Memproses data ke-4000 dari 5000 (Total attempts: 4000)...
Memproses data ke-4500 dari 5000 (Total attempts: 4500)...
Memproses data ke-5000 dari 5000 (Total attempts: 5000)...

Total attempts untuk generate 5000 data: 5000
--- Generasi Data Selesai ---
Distribusi Tipe Sisi (Counts):
- equilateral: 1865
- isosceles: 1863
- scalene: 1272

Distribusi Tipe Sudut (Counts):
- equiangular: 1865
- acute: 1850
- right: 615
- obtuse: 670

Data telah disimpan di triangle_revised_5k.csv
